In [ ]:
#fn = '/Users/savoie/projects/monthly_sea_ice_extent_tools/source/data/csvify/Dec/N_12_area.txt'
fn = '/Users/savoie/projects/monthly_sea_ice_extent_tools/source/data/csvify/Jun/N_06_area.txt'

In [ ]:
%matplotlib inline
import matplotlib.pylab
import pandas as pd
import numpy as np
import matplotlib as mpl
import datetime as dt
from scipy.stats import linregress
pd.options.display.mpl_style = 'default'


In [ ]:
def slurp_csv(filename):
    data = pd.read_csv(filename,
                       error_bad_lines=False,
                       skipinitialspace=True,
                       delimiter='\s+' )
    data.rename(columns={'mo': 'month', "region":"hemisphere"}, inplace=True)
    return data.dropna()


In [ ]:
data = slurp_csv(fn)

Set missing data to None Type.

In [ ]:
data.loc[ data.extent <= 0, ['extent', 'area', 'data_type']] = None
data = data.convert_objects(convert_numeric=True)

In [ ]:
data.dtypes

In [ ]:
a = data.copy()

Create an column of Date Periods and set it as the index.

In [ ]:
a['dates']= [pd.Period(dt.date(int(x[0]), int(x[1]), 1), "M") for x in zip(a.values[:,0], a.values[:,1])]
a = a.set_index('dates')

In [ ]:
a.head()

create new dataframe for the (rank sorted stuff)

In [ ]:
b = pd.DataFrame(index=a.index)

In [ ]:
#filler
b[' => '] = " "

b['rank'] = pd.DataFrame(data=a['extent'].rank(ascending=1))#.rename(columns={'extent':'rank'})

In [ ]:

b['ranked_year'] = a['year']
b['ranked_extent'] = a['extent']

In [ ]:
b.sort('rank', ascending=True, inplace=True)

compute extent anomaly

In [ ]:
a['extent_anomaly'] = a.extent - a[(a.year >= 1981) & (a.year <= 2010)].extent.mean()

In [ ]:
a.head()

In [ ]:
# cram the two parts together 
a.reset_index().join(b.reset_index(drop=True)).head()

In [ ]:
with open('../output/test.csv', 'w') as fp:
    fp.write('This is line1\n')
    fp.write('This is line2\n')
    a.to_csv(fp, header=True, float_format="%3.2f", index=False )